<a href="https://colab.research.google.com/github/Thirant/Thirant/blob/main/Map1_CSV_Json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import json

def convert_csv_to_json(csv_filepath, json_filepath, stage_name="Default Stage Name"):
    """
    Converte um arquivo CSV de dados de tile para um arquivo JSON estruturado
    para um jogo TRPG.
    """
    map_data = {
        "stage_name": stage_name,
        "width": 0,
        "height": 0,
        "tiles": [],
        "player_starts": [], # Será populado se 'initial_occupant_team' for 'player'
        "enemy_placements": [] # Será populado se 'initial_occupant_team' for 'enemy'
    }

    max_x = -1
    max_y = -1

    try:
        with open(csv_filepath, mode='r', newline='', encoding='utf-8') as csv_file:
            csv_reader = csv.DictReader(csv_file) # Trata a primeira linha como cabeçalho

            for row_number, row in enumerate(csv_reader, start=1):
                try:
                    # --- Extração e Conversão de Dados do Tile ---
                    x = int(row["x"])
                    y = int(row["y"])

                    if x > max_x: max_x = x
                    if y > max_y: max_y = y

                    tile_info = {
                        "coords": [x, y],
                        "terrain": row["terrain_type"].strip(),
                        "height": int(row["height"]),
                        "cost": int(row["movement_cost"]),
                        "passable": row["is_passable"].strip().upper() == 'TRUE' or row["is_passable"].strip() == '1'
                    }
                    map_data["tiles"].append(tile_info)

                    # --- Processamento de Unidades Iniciais ---
                    occupant_id = row.get("initial_occupant_id", "").strip()
                    occupant_team = row.get("initial_occupant_team", "").strip().lower()

                    if occupant_id: # Se houver um ocupante
                        placement_info = {
                            "unit_id_ref": occupant_id,
                            "coords": [x, y]
                            # Você pode adicionar mais infos aqui se necessário, como "facing_direction"
                        }
                        if occupant_team == "player":
                            map_data["player_starts"].append(placement_info)
                        elif occupant_team == "enemy":
                            map_data["enemy_placements"].append(placement_info)
                        else:
                            print(f"Aviso na linha {row_number+1}: Time de ocupante '{occupant_team}' não reconhecido para '{occupant_id}'. Ignorando.")

                except ValueError as e:
                    print(f"Erro de valor ao processar linha {row_number+1} do CSV: {row} - {e}")
                    print("Pulando esta linha.")
                    continue
                except KeyError as e:
                    print(f"Coluna faltando '{e}' na linha {row_number+1} do CSV: {row}")
                    print("Pulando esta linha.")
                    continue
                except Exception as e:
                    print(f"Erro inesperado ao processar linha {row_number+1}: {row} - {e}")
                    print("Pulando esta linha.")
                    continue

        if max_x == -1 or max_y == -1 : # Se nenhum tile foi lido
            if not map_data["tiles"]: # Se o array de tiles está vazio
                print("Nenhum dado de tile válido encontrado no CSV.")
                return False

        map_data["width"] = max_x + 1
        map_data["height"] = max_y + 1

        # Adicionar objetivos (exemplo fixo, pode vir de outro CSV ou ser mais dinâmico)
        map_data["objectives"] = [
            { "type": "DEFEAT_ALL_ENEMIES" }
        ]

        # Escrever o arquivo JSON
        with open(json_filepath, mode='w', encoding='utf-8') as json_file:
            json.dump(map_data, json_file, indent=2) # indent=2 para formatação legível

        print(f"Arquivo JSON '{json_filepath}' gerado com sucesso!")
        return True

    except FileNotFoundError:
        print(f"Erro: Arquivo CSV '{csv_filepath}' não encontrado.")
        return False
    except Exception as e:
        print(f"Um erro geral ocorreu: {e}")
        return False

# --- Como usar o script ---
if __name__ == "__main__":
    # Configure os caminhos dos arquivos aqui
    csv_input_path = "stage1_tiles.csv"  # Mude para o nome do seu arquivo CSV
    json_output_path = "stage1_map.json" # Nome do arquivo JSON de saída
    current_stage_name = "Floresta Assombrada"    # Nome do estágio

    # Crie um arquivo CSV de exemplo chamado 'stage1_tiles.csv' no mesmo diretório
    # que este script Python, com o conteúdo de exemplo fornecido acima, ou seus próprios dados.

    # Exemplo de como criar um CSV de teste se ele não existir:
    try:
        with open(csv_input_path, 'r') as f:
            pass # Arquivo existe
    except FileNotFoundError:
        print(f"Criando arquivo CSV de exemplo '{csv_input_path}'...")
        example_csv_content = """x,y,terrain_type,height,movement_cost,is_passable,initial_occupant_id,initial_occupant_team
0,0,grass,0,1,TRUE,,
1,0,grass,0,1,TRUE,player_hero,player
2,0,water,-1,99,FALSE,,
0,1,rock,1,2,TRUE,goblin_grunt,enemy
1,1,grass,0,1,TRUE,player_mage,player
2,1,rock,1,2,TRUE,goblin_archer,enemy
0,2,sand,0,2,TRUE,,
1,2,sand,0,2,TRUE,,
2,2,grass,0,1,TRUE,,
"""
        with open(csv_input_path, 'w', newline='', encoding='utf-8') as f:
            f.write(example_csv_content)

    if convert_csv_to_json(csv_input_path, json_output_path, current_stage_name):
        print("Conversão concluída.")
    else:
        print("Conversão falhou.")

Criando arquivo CSV de exemplo 'stage1_tiles.csv'...
Arquivo JSON 'stage1_map.json' gerado com sucesso!
Conversão concluída.
